In [2]:
import numpy as np
import pandas as pd

In [3]:
# Reading data
df = pd.read_csv('../data/raw/EURGBP.csv')
print(df.shape)
df.head()

(1035, 10)


,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,target
0,2000-01-03,6.281,633.000,6.235,6.272,NaN,NaN,NaN,NaN,0
1,2000-01-10,6.271,629.000,6.181,6.197,NaN,NaN,NaN,NaN,0
2,2000-01-17,0.620,6.225,6.072,609.000,NaN,NaN,NaN,NaN,1
3,2000-01-24,6.091,61.271,599.000,6.027,NaN,NaN,NaN,NaN,1
4,2000-01-31,6.021,6.267,598.000,6.158,NaN,NaN,NaN,NaN,0
